## Kauppalehti login & password

In [1]:
import json 
# Edit secrets.json file with your login details
with open('secrets.json', 'r') as file:
    secrets = json.load(file)
    LOGIN_EMAIL = secrets['KAUPPALEHTI_LOGIN_EMAIL']
    LOGIN_PASSWORD = secrets['KAUPPALEHTI_LOGIN_PASSWORD']

## Libraries

In [2]:
import numpy as np
import pandas as pd
import time
import os
import datetime

In [3]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


## Functions

'23.11.2023'

In [46]:
def fetch_kauppalehti_stocks(exchange, year=None):
    url_exchange = 'https://www.kauppalehti.fi/porssi/kurssit/{exchange}/historia'.format(exchange=exchange)

    driver.get(url_exchange)
    time.sleep(1)
    cal = driver.find_element(By.XPATH, '//div[@class="react-datepicker__input-container"]/input')
    cal.click()
    time.sleep(1)
    cal.send_keys(Keys.CONTROL + "a")
    time.sleep(0.5)
    if not year is None:
        cal.send_keys("01.01.{}".format(year))
    else:
        cal.send_keys(datetime.datetime.now().strftime("%d.%m.%Y"))
    time.sleep(0.5)

    html = driver.find_element(By.TAG_NAME, 'html')
    for i in range(10):
        html.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)

    stocks_add = {}
    stocks_ahref = driver.find_elements(By.XPATH, '//a[@class="ValueOutput__Link-tmcl-__sc-18ro7st-3 htvErj"]')
    stocks_add.update({stock.text: stock.get_attribute('href') for stock in stocks_ahref})
    stocks_ahref = driver.find_elements(By.XPATH, '//a[@class="ValueOutput__Link-tmcl-__sc-18ro7st-3 htKUiM"]')
    stocks_add.update({stock.text: stock.get_attribute('href') for stock in stocks_ahref})

    return stocks_add

In [5]:
class DownloadError(Exception):
    pass

In [6]:
def fetch_kauppalehti_prices(stock, download_path):
    stock_url = 'https://www.kauppalehti.fi/porssi/porssikurssit/osake/{stock}/kurssihistoria'.format(stock=stock)
    download_csv = os.path.join(download_path, 'kurssihistoria.csv')
    stock_csv = os.path.join(download_path, '{name}.csv'.format(name=stock))

    if not os.path.exists(stock_csv) or datetime.datetime.fromtimestamp(os.path.getmtime(stock_csv)).date() < datetime.date.today():
        if os.path.exists(download_csv):
            os.remove(download_csv)
        if os.path.exists(stock_csv):
            os.remove(stock_csv)

        driver.get(stock_url)
        time.sleep(1)

        #ytd = driver.find_element(By.XPATH, '//div[@class="timespan-selector-container undefined"]/button[text()="10 v"]')
        #ytd.click()
        #time.sleep(3)

        # set start date 1.1.2013
        cal = driver.find_element(By.XPATH, '//div[@class="react-datepicker__input-container"]/input')
        cal.click()
        time.sleep(1)
        cal.send_keys(Keys.CONTROL + "a")
        time.sleep(0.5)
        cal.send_keys("01.01.2013")
        time.sleep(1)
        cal = driver.find_element(By.XPATH, '//div[@class="react-datepicker__week"]//div[2]')
        cal.click()
        time.sleep(3)

        # download
        submit = driver.find_element(By.XPATH, '//div[@class="excel-container"]/button')
        submit.click()
        time.sleep(1)

        if os.path.exists(download_csv):
            os.rename(download_csv, stock_csv)
        else:
            raise DownloadError("Download CSV failed for {url}".format(url=stock_url))

In [7]:
def fetch_kauppalehti_dividends(download_path):
    download_csv = os.path.join(download_path, 'Osinkohistoria.csv')
    
    if not os.path.exists(download_csv) or datetime.datetime.fromtimestamp(os.path.getmtime(download_csv)).date() < datetime.date.today():
        if os.path.exists(download_csv):
            os.remove(download_csv)
            
        dividend_url = 'https://www.kauppalehti.fi/porssi/osinkohistoria/'
        driver.get(dividend_url)

        #ytd = driver.find_element(By.XPATH, '//div[@class="dividend-button-container"]/button[text()="10 v"]')
        #ytd.click()

        # set start date 1.1.2013
        cal = driver.find_element(By.XPATH, '//div[@class="react-datepicker__input-container"]/input')
        cal.click()
        time.sleep(0.5)
        cal.send_keys(Keys.CONTROL + "a")
        cal.send_keys("01.01.2013")
        time.sleep(0.5)
        cal.send_keys(Keys.ENTER)

        # download
        dl = driver.find_element(By.XPATH, '//div[@class="StockListActionHeader__ActionButtons-tmcl-__sc-1x2rtef-1 dUaQZW"]/button[2]')
        dl.click()
        time.sleep(1)

        if not os.path.exists(download_csv):
            raise DownloadError("Download CSV failed for {url}".format(url=dividend_url))

In [8]:
def get_total_return(download_path, key_file):
    # Kauppalehti ticker to stock name
    ticker_name = pd.read_csv(key_file).set_index('ticker')['name']
    
    # Read dividends
    fn_dividends = os.path.join(download_path,'Osinkohistoria.csv')
    dividends = pd.read_csv(fn_dividends, sep=";", decimal=",", parse_dates=[1], usecols=[0,1,2])
    dividends.rename(columns={"Irtoamispäivä": "Päivämäärä", "Voitonjako, €":"Osinko"}, inplace=True)
    dividends = dividends.groupby(['Osake', 'Päivämäärä'], as_index=False)['Osinko'].sum()
    dividends = dividends.sort_values(['Osake', 'Päivämäärä']).drop_duplicates()
    #dividends.head()
    
    # Read stock prices
    price = {}
    for filename in os.listdir('dls'):
        if filename.endswith(".csv") and filename != 'Osinkohistoria.csv': 
            stock = ticker_name[os.path.splitext(filename)[0]]
            stock_df = pd.read_csv(os.path.join('dls', filename), sep=';', decimal=',', parse_dates=[0])
            stock_df = stock_df.loc[~stock_df['Päivämäärä'].duplicated(keep='first'),]
            #stock_df['Päätöskurssi'].fillna(method='ffill', inplace=True)
            price[stock] = stock_df
    prices = pd.concat(price, axis=0, names=['Osake', 'n']).reset_index()
    prices = prices[['Osake', 'Päivämäärä', 'Vaihto €', 'Ylin', 'Alin', 'Keskimäärin', 'Päätöskurssi']]
    # some fixes
    prices.dropna(axis=0, subset=['Päätöskurssi'], inplace=True) 
    prices.drop(index=prices[prices['Osake'] == 'Efore Uudet 2018'].index, inplace=True)
    #prices.head()
    
    # Calculate total return
    returns = prices.merge(dividends, on=['Osake', 'Päivämäärä'], how='left')
    returns.sort_values(['Osake', 'Päivämäärä'], inplace=True)
    returns['Osinko'].fillna(0.0, inplace=True)
    returns['Osakemäärä'] = 1 + returns['Osinko'] / returns['Päätöskurssi']
    returns['Osakemäärä'] = returns.groupby('Osake')['Osakemäärä'].cumprod()
    returns['Päätöskurssi + Osinko'] = returns['Päätöskurssi'] * returns['Osakemäärä']
    #returns.head()
    
    # Save
    returns = returns[(returns['Päivämäärä'] >= pd.Timestamp('2013-01-01'))]# (returns['Päivämäärä'] < pd.Timestamp('2023-01-01'))
    returns = returns[['Osake', 'Päivämäärä', 'Vaihto €', 'Ylin', 'Alin', 'Keskimäärin', 'Päätöskurssi', 'Päätöskurssi + Osinko']]
    returns.to_csv("prices.csv", index=False, header=True, date_format='%Y-%m-%d', float_format='%.4f')

## Initialize selenium

In [36]:
download_path = os.path.join(os.getcwd(), 'dls')
webdriver_path = 'chromedriver.exe'

In [37]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : download_path}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(webdriver_path, chrome_options=chrome_options)

C:\Users\mvil4\AppData\Local\Temp\ipykernel_11540\506388869.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(webdriver_path, chrome_options=chrome_options)
C:\Users\mvil4\AppData\Local\Temp\ipykernel_11540\506388869.py:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(webdriver_path, chrome_options=chrome_options)


## Kauppalehti login

In [38]:
driver.get('https://www.kauppalehti.fi')

In [39]:
login_button = driver.find_element(By.XPATH, '//button[@aria-label="Kirjaudu"]')
login_button.click()

In [40]:
driver.find_element('id', 'alma-tunnus-username').send_keys(LOGIN_EMAIL)
driver.find_element('id', 'alma-tunnus-password').send_keys(LOGIN_PASSWORD + Keys.RETURN)

### Kauppalehti names

In [26]:
stocks = {}
for exchange in ['XHEL', 'FNFI']:
    print(exchange)
    for year in range(2013, datetime.date.today().year+1):
        print(year, end=': ')
        stocks_add = fetch_kauppalehti_stocks(exchange, year)
        stocks.update(stocks_add)
        print(len(stocks_add), "total")
    print("now", end=': ')
    stocks_add = fetch_kauppalehti_stocks(exchange)
    stocks.update(stocks_add)
    print(len(stocks_add), "total")
# This needs to be run more rarely ...

XHEL
2013: 124 total
2014: 130 total
2015: 136 total
2016: 143 total
2017: 147 total
2018: 146 total
2019: 151 total
2020: 147 total
2021: 144 total
2022: 147 total
2023: 146 total
FNFI
2013: 1 total
2014: 1 total
2015: 4 total
2016: 9 total
2017: 12 total
2018: 16 total
2019: 22 total
2020: 25 total
2021: 28 total
2022: 48 total
2023: 51 total


In [52]:
stocks_df = pd.DataFrame([(stock,url.split('/')[-1]) for stock, url in stocks.items()], columns=['name', 'ticker'])
stocks_df.loc[stocks_df['ticker'] == 'NDA%20FI', 'ticker'] = 'NDA FI'
stocks_df.to_csv('key_kauppalehti.csv', index=False)

In [53]:
stocks_df.head()

,name,ticker
0,Afarak Group,AFAGR
1,Affecto,AFE1V
2,Ahlstrom,AHL1V
3,Ahtium,AHTIUM
4,Aktia Pankki A,AKTIA


In [54]:
stocks_df = pd.read_csv('key_kauppalehti.csv')

### Kauppalehti price history

In [30]:
for i, stock in enumerate(stocks_df['ticker']):
    print("{}/{}".format(i+1, len(stocks_df)), stock)
    try:
        fetch_kauppalehti_prices(stock, download_path)
    except DownloadError as e:
        print(e)

1/234 AFAGR
2/234 AFE1V
3/234 AHL1V
4/234 AHTIUM
5/234 AKTIA
6/234 AKTRV
7/234 ALMA
8/234 AMEAS
9/234 APETIT
10/234 ASPO
11/234 ACG1V
12/234 ATRAV
13/234 BAS1V
14/234 BIOBV
15/234 BTH1V
16/234 BITTI
17/234 BOREO
18/234 CAPMAN
19/234 CGCBV
20/234 CTY1S
21/234 CTH1V
22/234 CTL1V
23/234 CRA1V
24/234 DIGIA
25/234 DIGIGR
26/234 DOV1V
27/234 ELEAV
28/234 ELISA
29/234 ENEDO
30/234 EQV1V
31/234 ERIBR
32/234 ETTE
33/234 EXL1V
34/234 FIA1S
35/234 FSKRS
36/234 FORTUM
37/234 GLA1V
38/234 HKSAV
39/234 HONBS
40/234 HUH1V
41/234 ILKKA1
42/234 ILKKA2
43/234 ICP1V
44/234 IFA1V
45/234 INVEST
46/234 KEMIRA
47/234 KSLAV
48/234 KESKOA
49/234 KESKOB
50/234 KELAS
51/234 KHG
52/234 KNEBV
53/234 KCR
54/234 LAT1V
55/234 LEM1S
56/234 MEKKO
57/234 MARAS
58/234 METSO
59/234 METSA
60/234 METSB
61/234 NELES
62/234 NESTE
63/234 NOKIA
64/234 TYRES
65/234 NDA FI
66/234 NORVE
67/234 NLG1V
68/234 OKM1V
69/234 OLVAS
70/234 OKDAV
71/234 OKDBV
72/234 ORNAV
73/234 ORNBV
74/234 OUT1V
75/234 PNA1V
76/234 PKC1V
77/234 PON1V
78/

### Kauppalehti dividend history 

In [31]:
fetch_kauppalehti_dividends(download_path)

### Kauppalehti total return

In [57]:
get_total_return(download_path, 'key_kauppalehti.csv')

### Nasdaq

In [111]:
driver.get("http://www.nasdaqomxnordic.com/index/index_info?Instrument=FI0008900006&name=OMX%20Helsinki_PI")
el = driver.find_elements(By.XPATH, '//table[@id="sharesInIndexTable"]/tbody/tr')
tickers_omxh = [e.get_attribute('title').split('-')[0].strip() for e in el]

In [112]:
driver.get("http://www.nasdaqomxnordic.com/index/index_info?Instrument=SE0007551361&name=First%20North%20Finland%20EUR%20PI")
el = driver.find_elements(By.XPATH, '//table[@id="sharesInIndexTable"]/tbody/tr')
tickers_fnfi = [e.get_attribute('title').split('-')[0].strip() for e in el]

In [113]:
tickers = tickers_omxh + tickers_fnfi

In [125]:
# Check that all (current) tickers were succesfully fetched from kauppalehti
set(tickers).difference(set(stocks_df['ticker']))

set()

### Yahoo

In [126]:
import yfinance as yf

In [52]:
for i, ticker in enumerate(tickers):
    print(ticker, "{}/{}".format(i+1, len(tickers)))
    stock_df = yf.Ticker("{ticker}.HE".format(ticker='NDA-FI' if ticker == 'NDA FI' else ticker))
    hist = stock_df.history(start='2013-01-01', end=datetime.datetime.now().strftime('%Y-%m-%d'), period="max")
    hist.to_csv(os.path.join('yahoo', '{stock}.csv'.format(stock=ticker)), float_format='%.3f')
    time.sleep(10)

ACG1V 1/192
AFAGR 2/192
AKTIA 3/192
ALBAV 4/192
ALBBV 5/192
ALMA 6/192
ANORA 7/192
APETIT 8/192
ASPO 9/192
ATRAV 10/192
BIOBV 11/192
BITTI 12/192
BOREO 13/192
CAPMAN 14/192
CAV1V 15/192
CGCBV 16/192
CONSTI 17/192
CTH1V 18/192
CTY1S 19/192
DIGIA 20/192
DIGIGR 21/192
DOV1V 22/192
EEZY 23/192
ELEAV 24/192
ELISA 25/192
ENENTO 26/192
EQV1V 27/192
ESENSE 28/192
ETTE 29/192
EVLI 30/192
EXL1V 31/192
FELLOW 32/192
FIA1S 33/192
FORTUM 34/192
FSECURE 35/192
FSKRS 36/192
GLA1V 37/192
GOFORE 38/192
HARVIA 39/192
HKSAV 40/192
HONBS 41/192
HUH1V 42/192
ICP1V 43/192
IFA1V 44/192
ILKKA2 45/192
INVEST 46/192
KAMUX 47/192
KCR 48/192
KELAS 49/192
KEMIRA 50/192
KESKOA 51/192
KESKOB 52/192
KNEBV 53/192
KOJAMO 54/192
KOSKI 55/192
KREATE 56/192
KSLAV 57/192
LAT1V 58/192
LEHTO 59/192
MARAS 60/192
MEKKO 61/192
METSA 62/192
METSB 63/192
MOCORP 64/192
MUSTI 65/192
NDA FI 66/192
NESTE 67/192
NIXU 68/192
NLG1V 69/192
NOHO 70/192
NOKIA 71/192
OKDAV 72/192
OKDBV 73/192
OLVAS 74/192
OMASP 75/192
OPTOMED 76/192
ORNAV 7